# Team SNPS

# Assignment-9

In [1]:
import csv
import pandas as pd
import numpy as np
from DATA225utils import make_connection, dataframe_query 

ModuleNotFoundError: No module named 'DATA225utils'

In [ ]:
conn_analytical = make_connection(config_file = 'snps_wh.ini')
cursor_analytical = conn_analytical.cursor()

### Create at least two queries per fact table that join with the dimension tables.
#### o Describe the query in English.
#### o Write and execute the SQL.

## PlayerStats Fact Table

## Query 1:

#### In this query we are trying to retrieve the player statistics such as total_points, rebounds, and assists for each player in various seasons, along with the season year and the player name.

    -The SQL query displays the player statistics (total_points, rebounds, and assists) with player names for various 
    seasons, sorted by season. 
    
    -To obtain the necessary information, the query connects fact and dimension tables including player_stats, game, player, 
    and calendar. 
    
    -To obtain the player name, it first combines the player table and the player_stats table using the player_key column. 
    Then, in order to obtain season information, it links the game table to the player_stats table through the game_key 
    column along with the calendar table to the game table of the date_key column.
    
    -The query joins the tables and then summarises the statistics of each player's total_points, rebounds, and assists 
    for every season using the SUM() function. The GROUP BY and ORDER BY clauses are used to group the data by season 
    along with player name and to arrange the outcomes by season. 
    
    
    The final output of the query is a table with the player's name, season, and the number of points, rebounds, and assists they have received in each of their seasons.

In [ ]:
sql1 = ('''
      SELECT calendar.yr as season, player.player_name, SUM(player_stats.points) as total_points, SUM(player_stats.rebounds) as total_rebounds, SUM(player_stats.assists) as total_assists
FROM player_stats
JOIN player ON player_stats.player_key = player.player_key
JOIN game ON player_stats.game_key = game.game_key
JOIN calendar ON game.date_key = calendar.date_key
GROUP BY calendar.yr, player.player_name
ORDER BY calendar.yr;

        ''')
dataframe_query(conn_analytical, sql1)[1]

## Query 2:

#### In this query we are trying to retrieve the total number of games played by each player in each season, along with the player name and season year.

    -The purpose of the query is to return information on each player's total number of games played during each season.
    
    -To obtain the player name, we first join the player_stats table to the player table using the player_key. 
    
    -The games played by each player are then obtained by joining the game table to the player_stats table based on the game 
     key.     
    
    -The year of the season for each game is then obtained by joining the calendar table to the game table using the date 
    key. 
    
    -In order to determine how many games each player played during each season, we categorize the stats by player name in 
     addition to season year. 
    
    -The results are then filtered by only including players who have played at least 50 games in a season, and they are 
    then arranged in descending order by the number of games played by each player overall during each season.

In [ ]:
sql2 = ('''
        SELECT player.player_name, calendar.yr as season, COUNT(DISTINCT game.game_key) as total_games_played
        FROM player_stats
        JOIN player ON player_stats.player_key = player.player_key
        JOIN game ON player_stats.game_key = game.game_key
        JOIN calendar ON game.date_key = calendar.date_key
        GROUP BY player.player_name, calendar.yr
        HAVING COUNT(DISTINCT game.game_key) >= 50
        ORDER BY total_games_played DESC;

        ''')
dataframe_query(conn_analytical, sql2)[1]

## Standings Fact Table

## Query 1

Displays the list of teams that won in the second quatar of the year 2005 at the West conference , along with the city they are from and corresponding win percentage.

In [ ]:
_, df = dataframe_query(conn_analytical,
                        
        """
            select t.team_name, t.city, t.conference,s.win_pct, c.yr
            FROM team t, calendar c, standings s
            where t.team_key = s.team_key
            and c.date_key = s.date_key
            and c.yr = 2005
            and c.qtr = 2
            and t.conference = 'West';
        """ )

df

## Query 2

Displays the total total number of wins, of each team that won in the year 2005 , along with the states they are from, in descending order.

In [ ]:
_, df = dataframe_query(conn_analytical,
                        
        """
            select sum(s.wins) as Total_Wins, t.team_name, t.state
            FROM team t, calendar c, standings s
            where t.team_key = s.team_key
            and c.date_key = s.date_key
            and c.yr = 2005
            GROUP BY t.team_name, t.state
            ORDER BY Total_Wins DESC;
        """ )
df

## TeamStats Fact Table

## Query 1

#### Which team scored the most points in the 'Eastern Conference'? 

In [ ]:
_, df = dataframe_query(conn_analytical,               
        """
            SELECT t.team_Name, SUM(f.Points) AS Total_Points 
            FROM Fact_Table f 
            JOIN Team t ON f.team_key = t.team_key
            WHERE t.conference = 'Eastern Conference'
            GROUP BY t.team_Name
            ORDER BY Total_Points DESC
            LIMIT 1
        """ )
df

In [ ]:
_, df = dataframe_query(conn_analytical,               
        """
            SELECT t.team_name, 
                   SUM(CASE WHEN f.Points > o.Away_Score THEN 1 ELSE 0 END) AS Wins,
                   SUM(CASE WHEN f.Points < o.Away_Score THEN 1 ELSE 0 END) AS Losses 
            FROM Fact_Table f 
            JOIN Team t ON f.Team_key = t.Team_key
            JOIN Game o ON f.Game_key = o.Game_key
            WHERE t.team_name = 'Los Angeles Lakers' 
            GROUP BY t.team_name
        """ )
df

In [ ]:
cursor_analytical.close()
conn_analytical.close()